In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

DOMAIN = 'spouse'

db_dict = {
    'test':   'postgres://localhost:5432/babble_model_unittest',
    'spouse': 'postgres://localhost:5432/babble_model_spouse',
    'cdr':    'postgres://localhost:5432/babble_model_cdr',
}

os.environ['SNORKELDB'] = db_dict[DOMAIN]

In [3]:
from snorkel import SnorkelSession
session = SnorkelSession()

In [4]:
from snorkel.contrib.babble.models import configuration

config = configuration
config['domain'] = DOMAIN
config['source'] = 'nl'
config['parallelism'] = 1
config['majority_vote'] = True
config['num_epochs'] = 50
config['num_search'] = 10

In [5]:
from snorkel.models import candidate_subclass
Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

In [12]:
from tutorials.babble.spouse import SpouseModel
sm = SpouseModel(session, Spouse, config)

In [7]:
# %time sm.parse()

In [8]:
# %time sm.extract()

In [9]:
# %time sm.load_gold()

In [10]:
from tutorials.babble.data import MTurkHelper
output_csv_path= os.environ['SNORKELHOME'] + '/tutorials/babble/data/spouse/mturk_spouse_01_out.csv'
candidates = session.query(Spouse).all()
helper = MTurkHelper(candidates, num_hits=25)
explanations = helper.postprocess(output_csv_path, candidates=candidates, verbose=False)
user_lists = {}

Num HITs unique: 25
Num HITs total: 75
Unanimous: 61
Majority: 14
Bad: 25
Total explanations: 211


In [ ]:
%time sm.babble(explanations, user_lists=user_lists)

In [ ]:
# sm.use_intro_lfs()

In [ ]:
%time sm.label()

In [ ]:
%time sm.supervise()

In [ ]:
%time sm.classify()